In [8]:
import refexps
from os import listdir, path
from itertools import combinations
from collections import defaultdict
import numpy as np
from scipy import stats

dataset_dir = 'Dataset/Push_to_Talk_Corpus/RefExp'
success_dir = 'Dataset/Push_to_Talk_Corpus/Success'

In [9]:
move_level_data = defaultdict(dict)
move_level_align = {'FTT': [], 'PTT': []}
move_level_coherence = {'FTT': [], 'PTT': []}
success_types = set()
playermove_level_align_speaker = {'FTT': defaultdict(list), 'PTT': defaultdict(list)} 
playermove_level_coherence_speaker= {'FTT': defaultdict(list), 'PTT': defaultdict(list)}  

for dialogue in listdir(dataset_dir):
    #extract dialogue type
    dial_type = dialogue[:3]
    success_file = dialogue.replace('refexps', 'movesuccess')
    success_file = path.join(success_dir, success_file)
    dialogue = path.join(dataset_dir, dialogue)
    #load referring expressions data
    dialogue = refexps.load_data(dialogue)
    success_moves = refexps.load_data(success_file)
    #move level
    dialogue_move = refexps.move_level(dialogue)
    #iterate over moves
    for move in dialogue_move:
        refs = move[1]
        #iterate over objects
        for obj in refs:
            #iterate over objects
            refobj = refs[obj]
            #SIMILARITY OF LEXICAL CONTENT INTO
            sim_align = []
            sim_align_speaker = defaultdict(list)
            sim_coherence_speaker = defaultdict(list)
            sim_coherence = []
            for ref in refobj:
                speaker = ref[1]
                sim = refexps.lex_sim(ref, refexps.get_previous(ref, refobj))
                if sim != 'undefined':
                    sim_align.append(sim)
                    sim_align_speaker[speaker].append(sim)
            sim_align = np.mean(sim_align)
            comb_ref = combinations(refobj,2)
            for comb in comb_ref:
                if comb[0] != comb[1]:
                    sim = refexps.lex_sim(comb[0], comb[1])
                    if sim != 'undefined':
                        sim_coherence.append(sim)
                        sim_coherence_speaker[comb[0][1]].append(sim)
                        sim_coherence_speaker[comb[1][1]].append(sim)
            sim_coherence = np.mean(sim_coherence)
            #movelevel coherence and alignment (for each object)
            if not np.isnan(sim_coherence):
                move_level_coherence[dial_type].append(sim_coherence)
            if not np.isnan(sim_align):
                move_level_align[dial_type].append(sim_align)
            for s in sim_align_speaker:
                if not np.isnan(np.mean(sim_align_speaker[s])):
                    playermove_level_align_speaker[dial_type][s].append(np.mean(sim_align_speaker[s]))
            for s in sim_coherence_speaker:
                if not np.isnan(np.mean(sim_coherence_speaker[s])) :
                    playermove_level_coherence_speaker[dial_type][s].append(np.mean(sim_coherence_speaker[s]))
        values = [sim_coherence, sim_align, sim_align_speaker['e-utts'], sim_align_speaker['p-utts'], sim_coherence_speaker['e-utts'], sim_coherence_speaker['p-utts'] ]
        move_level_data[move[0]]['values'] = values
        success = success_moves[move[0]]
        move_level_data[move[0]]['success'] = success
move_level_align_all = []
move_level_coherence_all = []

playermove_level_align_speaker_all = defaultdict(list)
playermove_level_coherence_speaker_all = defaultdict(list)

results_move = {'FTT': {}, 'PTT': {}, 'ALL': defaultdict(list)}

for dial_type in playermove_level_coherence_speaker:
    move_level_align_all += move_level_align[dial_type]
    move_level_coherence_all += move_level_coherence[dial_type]
    playermove_level_coherence = []
    for s in playermove_level_coherence_speaker[dial_type]:
        playermove_level_coherence += playermove_level_coherence_speaker[dial_type][s]
        playermove_level_coherence_speaker_all[s] += playermove_level_coherence_speaker[dial_type][s]
        results_move[dial_type]['Move coherence '+ s] = playermove_level_coherence_speaker[dial_type][s]
        results_move['ALL']['Move coherence '+ s].extend(playermove_level_coherence_speaker[dial_type][s])
    results_move[dial_type]['Move players coherence'] = playermove_level_coherence
    results_move[dial_type]['Move coherence'] = move_level_coherence[dial_type]
    results_move['ALL']['Move players coherence'].extend(playermove_level_coherence)
    results_move['ALL']['Move coherence'].extend(move_level_coherence[dial_type])
    playermove_level_align = []
    for s in playermove_level_align_speaker[dial_type]:
        playermove_level_align += playermove_level_align_speaker[dial_type][s]
        playermove_level_align_speaker_all[s] += playermove_level_align_speaker[dial_type][s]
        results_move[dial_type]['Move alignment '+ s] = playermove_level_align_speaker[dial_type][s]
        results_move['ALL']['Move alignment '+ s].extend(playermove_level_align_speaker[dial_type][s])
    results_move['ALL']['Move players alignment'].extend(playermove_level_align)
    results_move['ALL']['Move alignment'].extend(move_level_align[dial_type])
    results_move[dial_type]['Move players alignment'] = playermove_level_align
    results_move[dial_type]['Move alignment'] = move_level_align[dial_type]
for d_type in results_move:
    print d_type
    for value in results_move[d_type]:
        dataset = results_move[d_type][value]
        print '\t'+ value + ': '
        print '\tMean: '+ str(np.mean(dataset)) +'\tStandard deviation: '+ str(np.std(dataset))
        print '\tMax value: ' + str(max(dataset)) + '\tMin value: '+ str(min(dataset))

ALL
	Move coherence: 
	Mean: 0.203856682561	Standard deviation: 0.288291555541
	Max value: 1.0	Min value: 0.0
	Move alignment e-utts: 
	Mean: 0.0790351757901	Standard deviation: 0.221118532935
	Max value: 1.0	Min value: 0.0
	Move alignment p-utts: 
	Mean: 0.0219336742922	Standard deviation: 0.120763302484
	Max value: 1.0	Min value: 0.0
	Move players coherence: 
	Mean: 0.193743026017	Standard deviation: 0.286374448479
	Max value: 1.0	Min value: 0.0
	Move players alignment: 
	Mean: 0.0399220309838	Standard deviation: 0.161541701766
	Max value: 1.0	Min value: 0.0
	Move coherence p-utts: 
	Mean: 0.197583576222	Standard deviation: 0.283689545598
	Max value: 1.0	Min value: 0.0
	Move coherence e-utts: 
	Mean: 0.18672254503	Standard deviation: 0.291087500108
	Max value: 1.0	Min value: 0.0
	Move alignment: 
	Mean: 0.0235097206455	Standard deviation: 0.0902002930845
	Max value: 0.666666666667	Min value: 0.0
PTT
	Move coherence: 
	Mean: 0.233074821332	Standard deviation: 0.297303964269
	Max value

ALL
In general, higher values than at the game level

* Move coherence: some variation
* Move alignment:
    e-utts:more aligned, variation
    p-utts: less aligned, much variation
* Move players coherence: some variation
* Move players alignment: much variation 
* Move coherence:
    e-utts: more coherent, some variation
    p-utts: more coherent, some variation
* Move alignment: much variation

In [34]:
values = results_move['FTT'].keys()
print 'DIFFERENCES PTT AND FTT'
for value in values:
    ttest =  stats.ttest_ind(results_move['FTT'][value],results_move['PTT'][value] )
    if ttest[1] < 0.05:
        print value
        print str(ttest)+'\tSignificantly different'
print 'DIFFERENCES E-UTTS AND P-UTTS'
ttest = stats.ttest_ind(results_move['FTT']['Move coherence e-utts'], results_move['FTT']['Move coherence p-utts'])
if ttest[1] < 0.05:
    print 'Move coherence speakers FTT: '+ str(ttest) + '\tSignificantly different'
ttest = stats.ttest_ind(results_move['PTT']['Move coherence e-utts'], results_move['PTT']['Move coherence p-utts'])
if ttest[1] < 0.05:
    print 'Move coherence speakers PTT: '+ str(ttest) + '\tSignificantly different'
ttest = stats.ttest_ind(results_move['ALL']['Move coherence e-utts'], results_move['ALL']['Move coherence p-utts'])
if ttest[1] < 0.05:
    print 'Move coherence speakers ALL: '+ str(ttest) + '\tSignificantly different'
ttest = stats.ttest_ind(results_move['FTT']['Move alignment e-utts'], results_move['FTT']['Move alignment p-utts'])
if ttest[1] < 0.05:
    print 'Move alignment speakers FTT: '+ str(ttest) + '\tSignificantly different'
ttest = stats.ttest_ind(results_move['PTT']['Move alignment e-utts'], results_move['PTT']['Move alignment p-utts'])
if ttest[1] < 0.05:
    print 'Move alignment speakers PTT: '+ str(ttest) + '\tSignificantly different'
ttest = stats.ttest_ind(results_move['ALL']['Move alignment e-utts'], results_move['ALL']['Move alignment p-utts'])
if ttest[1] < 0.05:
    print 'Move alignment speakers ALL: '+ str(ttest) + '\tSignificantly different'

DIFFERENCES PTT AND FTT
Move coherence
Ttest_indResult(statistic=-2.2439970521064989, pvalue=0.025160299492474503)	Significantly different
Move alignment p-utts
Ttest_indResult(statistic=-2.5954454003863954, pvalue=0.0096095687315416716)	Significantly different
Move players coherence
Ttest_indResult(statistic=-3.1230839195442726, pvalue=0.0018425709759182993)	Significantly different
Move coherence p-utts
Ttest_indResult(statistic=-2.7256110690370514, pvalue=0.0065969136405423373)	Significantly different
DIFFERENCES E-UTTS AND P-UTTS
Move alignment speakers FTT: Ttest_indResult(statistic=6.3104630637888857, pvalue=4.8777898377212376e-10)	Significantly different
Move alignment speakers PTT: Ttest_indResult(statistic=2.6036750250607978, pvalue=0.0094835564673469933)	Significantly different
Move alignment speakers ALL: Ttest_indResult(statistic=6.351157590656868, pvalue=2.9908123380271225e-10)	Significantly different


DIFFERENCES PTT AND FTT

Move coherence: PTT more coherent

Move alignment p-utts: p-utts in PTT are more aligned

Move players coherence: PTT speakerds are more coherent 

Move coherence p-utts: PTTs more coherent

DIFFERENCES E-UTTS AND P-UTTS

Move alignment speakers FTT: e-utts more aligned

Move alignment speakers PTT: e-utts more aligned

Move alignment speakers ALL: e-utts more aligned

In [35]:
game_level_align = {'FTT': [], 'PTT': []}
game_level_coherence = {'FTT': [], 'PTT': []}
playergame_level_align_speaker = {'FTT': defaultdict(list), 'PTT': defaultdict(list)} 
playergame_level_coherence_speaker = {'FTT': defaultdict(list), 'PTT': defaultdict(list)} 

for dialogue in listdir(dataset_dir):
    dial_type = dialogue[:3]
    dialogue = refexps.load_data(path.join(dataset_dir, dialogue))
    dialogue_game = refexps.gamerun_level(dialogue)
    for obj in dialogue_game:
        refobj = dialogue_game[obj]
        sim_align = []
        sim_align_speaker = defaultdict(list)
        sim_coherence_speaker = defaultdict(list)
        sim_coherence = []
        for ref in refobj:
            speaker = ref[1]
            sim = refexps.lex_sim(ref, refexps.get_previous(ref, refobj))
            if sim != 'undefined':
                sim_align.append(sim)
                sim_align_speaker[speaker].append(sim)
        sim_align = np.mean(sim_align)
        for s in sim_align_speaker[speaker]:
            sim_align_speaker[speaker] = np.mean(sim_align_speaker[speaker])
        comb_ref = combinations(refobj,2)
        for comb in comb_ref:
            if comb[0] != comb[1]:
                sim = refexps.lex_sim(comb[0], comb[1])
                if sim != 'undefined':
                    sim_coherence.append(sim)
                    sim_coherence_speaker[comb[0][1]].append(sim)
                    sim_coherence_speaker[comb[1][1]].append(sim)
        sim_coherence = np.mean(sim_coherence)
        #gamelevel coherence and alignment (for each object)
        if not np.isnan(sim_coherence) :
            game_level_coherence[dial_type].append(sim_coherence)
        if not np.isnan(sim_align):
            game_level_align[dial_type].append(sim_align)
        for s in sim_align_speaker:
            if not np.isnan(np.mean(sim_align_speaker[s])):
                playergame_level_align_speaker[dial_type][s].append(np.mean(sim_align_speaker[s]))
        for s in sim_coherence_speaker:
            if not np.isnan(np.mean(sim_coherence_speaker[s])) :
                playergame_level_coherence_speaker[dial_type][s].append(np.mean(sim_coherence_speaker[s]))

game_level_align_all = []
game_level_coherence_all = []

playergame_level_align_speaker_all = defaultdict(list)
playergame_level_coherence_speaker_all = defaultdict(list)

results_game = {'FTT': {}, 'PTT': {}, 'ALL': defaultdict(list)}

for dial_type in playergame_level_coherence_speaker:
    game_level_align_all += game_level_align[dial_type]
    game_level_coherence_all += game_level_coherence[dial_type]
    playergame_level_coherence = []
    for s in playergame_level_coherence_speaker[dial_type]:
        playergame_level_coherence += playergame_level_coherence_speaker[dial_type][s]
        playergame_level_coherence_speaker_all[s] += playergame_level_coherence_speaker[dial_type][s]
        results_game[dial_type]['game coherence '+ s] = playergame_level_coherence_speaker[dial_type][s]
        results_game['ALL']['game coherence '+ s].extend(playergame_level_coherence_speaker[dial_type][s])
    results_game[dial_type]['game players coherence'] = playergame_level_coherence
    results_game[dial_type]['game coherence'] = game_level_coherence[dial_type]
    results_game['ALL']['game players coherence'].extend(playergame_level_coherence)
    results_game['ALL']['game coherence'].extend(game_level_coherence[dial_type])
    playergame_level_align = []
    for s in playergame_level_align_speaker[dial_type]:
        playergame_level_align += playergame_level_align_speaker[dial_type][s]
        playergame_level_align_speaker_all[s] += playergame_level_align_speaker[dial_type][s]
        results_game[dial_type]['game alignment '+ s] = playergame_level_align_speaker[dial_type][s]
        results_game['ALL']['game alignment '+ s].extend(playergame_level_align_speaker[dial_type][s])
    results_game['ALL']['game players alignment'].extend(playergame_level_align)
    results_game['ALL']['game alignment'].extend(game_level_align[dial_type])
    results_game[dial_type]['game players alignment'] = playergame_level_align
    results_game[dial_type]['game alignment'] = game_level_align[dial_type]
for d_type in results_game:
    print d_type
    for value in results_game[d_type]:
        dataset = results_game[d_type][value]
        print '\t'+ value + ': '
        print '\tMean: '+ str(np.mean(dataset)) +'\tStandard deviation: '+ str(np.std(dataset))
        print '\tMax value: ' + str(max(dataset)) + '\tMin value: '+ str(min(dataset))

ALL
	game players coherence: 
	Mean: 0.145461311759	Standard deviation: 0.19958087699
	Max value: 1.0	Min value: 0.0
	game coherence e-utts: 
	Mean: 0.131800781514	Standard deviation: 0.175287409655
	Max value: 1.0	Min value: 0.0
	game alignment: 
	Mean: 0.0246849331989	Standard deviation: 0.0714100450011
	Max value: 0.533333333333	Min value: 0.0
	game alignment p-utts: 
	Mean: 0.0184436472589	Standard deviation: 0.0779906992634
	Max value: 1.0	Min value: 0.0
	game coherence p-utts: 
	Mean: 0.152953612035	Standard deviation: 0.211349986719
	Max value: 1.0	Min value: 0.0
	game players alignment: 
	Mean: 0.0403250181937	Standard deviation: 0.128197562368
	Max value: 1.0	Min value: 0.0
	game coherence: 
	Mean: 0.15465113007	Standard deviation: 0.211361993896
	Max value: 1.0	Min value: 0.0
	game alignment e-utts: 
	Mean: 0.0870673509991	Standard deviation: 0.187991376545
	Max value: 1.0	Min value: 0.0
PTT
	game players coherence: 
	Mean: 0.178336791372	Standard deviation: 0.21879363613
	Ma

TO DO: 
    * overall coherence
    * overall alignment
    * coherence within refexps of one speaker
    
ALL: 
* game coherence: some variation
* Game players coherence: some variation 
* Game coherence:
    e-utts: less coherent; 
    p-utts: more coherent, more variation. 
    
* game alignment: variation! Max value: 0.53 
* game players alignment: much variation across speakers
* game alignment:
    e-utts: much more alignment and some variation
    p-utts: less alignment, but variation
    
In general, lower values for alignment then for coherence

In [36]:
values = results_game['FTT'].keys()
print 'DIFFERENCES PTT AND FTT'
for value in values:
    ttest =  stats.ttest_ind(results_game['FTT'][value],results_game['PTT'][value] )
    if ttest[1] < 0.05:
        print value
        print str(ttest)+'\tSignificantly different'
print 'DIFFERENCES E-UTTS AND P-UTTS'
ttest = stats.ttest_ind(results_game['FTT']['game coherence e-utts'], results_game['FTT']['game coherence p-utts'])
if ttest[1] < 0.05:
    print 'game coherence speakers FTT: '+ str(ttest) + '\tSignificantly different'
ttest = stats.ttest_ind(results_game['PTT']['game coherence e-utts'], results_game['PTT']['game coherence p-utts'])
if ttest[1] < 0.05:
    print 'game coherence speakers PTT: '+ str(ttest) + '\tSignificantly different'
ttest = stats.ttest_ind(results_game['ALL']['game coherence e-utts'], results_game['ALL']['game coherence p-utts'])
if ttest[1] < 0.05:
    print 'game coherence speakers ALL: '+ str(ttest) + '\tSignificantly different'
ttest = stats.ttest_ind(results_game['FTT']['game alignment e-utts'], results_game['FTT']['game alignment p-utts'])
if ttest[1] < 0.05:
    print 'game alignment speakers FTT: '+ str(ttest) + '\tSignificantly different'
ttest = stats.ttest_ind(results_game['PTT']['game alignment e-utts'], results_game['PTT']['game alignment p-utts'])
if ttest[1] < 0.05:
    print 'game alignment speakers PTT: '+ str(ttest) + '\tSignificantly different'
ttest = stats.ttest_ind(results_game['ALL']['game alignment e-utts'], results_game['ALL']['game alignment p-utts'])
if ttest[1] < 0.05:
    print 'game alignment speakers ALL: '+ str(ttest) + '\tSignificantly different'

DIFFERENCES PTT AND FTT
game players coherence
Ttest_indResult(statistic=-4.1814492536583234, pvalue=3.2919172495152311e-05)	Significantly different
game alignment p-utts
Ttest_indResult(statistic=-2.2542868534695777, pvalue=0.024610185793786356)	Significantly different
game coherence p-utts
Ttest_indResult(statistic=-3.9866566169393773, pvalue=7.8990056406735037e-05)	Significantly different
game coherence
Ttest_indResult(statistic=-3.8051731890530704, pvalue=0.00016240440966135096)	Significantly different
DIFFERENCES E-UTTS AND P-UTTS
game alignment speakers FTT: Ttest_indResult(statistic=6.1585262553998721, pvalue=1.8786149589784061e-09)	Significantly different
game alignment speakers PTT: Ttest_indResult(statistic=3.5587730756886904, pvalue=0.00042323415641905407)	Significantly different
game alignment speakers ALL: Ttest_indResult(statistic=6.9840773255961919, pvalue=6.426116264004366e-12)	Significantly different


DIFFERENCES PTT AND FTT

game players coherence: speakers in PTT are more coherent

game alignment p-utts: p-utts in PTT are more aligned

game coherence: in PTT the players are more coherent


DIFFERENCES E-UTTS AND P-UTTS
Move alignment speakers FTT: e-utts  more aligned
Move alignment speakers PTT: e-utts more aligned
Move alignment speakers ALL: e-utts more aligned

In [51]:
values = results_game['FTT'].keys()
games = results_game.keys()
count_move = 0
count = 0

print 'DIFFERENCES PTT AND FTT'
for value in values:
    for game in games:
        count += 1
        ttest =  stats.ttest_ind(results_game[game][value],results_move[game][value.replace('game', 'Move')] )
        if ttest[1] < 0.05:
            print value, game
            if np.mean(results_game[game][value]) > np.mean(results_move[game][value.replace('game', 'Move')]):
                print 'Game level is higher'
            else:
                print 'Move level is higher'
            print '\tSignificantly different'    

DIFFERENCES PTT AND FTT
game players coherence ALL
Move level is higher
	Significantly different
game players coherence PTT
Move level is higher
	Significantly different
game players coherence FTT
Move level is higher
	Significantly different
game coherence e-utts ALL
Move level is higher
	Significantly different
game coherence e-utts PTT
Move level is higher
	Significantly different
game coherence p-utts ALL
Move level is higher
	Significantly different
game coherence p-utts FTT
Move level is higher
	Significantly different
game coherence ALL
Move level is higher
	Significantly different
game coherence FTT
Move level is higher
	Significantly different


game players coherence and game coherence is always significantly higher at the move level